<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/162_LangGraph_Memory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

🧠 — memory in **LangGraph** is both simpler and more powerful than it might look at first. Let’s build you a good mental model.

---

# 🧠 Memory in LangGraph

At a high level:

* **Memory = information that persists across steps in the graph.**
* In LangGraph, *everything is state*. Memory is just part of your **State** object.
* Instead of hidden/magic memory (like in vanilla LangChain agents), you make memory **explicit** in your `State` definition.

This is why LangGraph feels very “transparent”: you can always see what your agent knows.

---

## 1. Memory Lives in the State

For example:

```python
from typing import TypedDict, List, Dict, Annotated
from langgraph.graph import add_messages

class AgentState(TypedDict):
    messages: Annotated[List[Dict], add_messages]  # memory of conversation
    goal: str
    plan: List[str]
    progress: List[str]
```

Here:

* `messages` is our memory store.
* The `add_messages` reducer makes sure new messages are **appended** instead of overwriting.

---

## 2. How Memory Updates

Every node that produces new info returns a **diff**:

```python
def user_message_node(state: AgentState) -> AgentState:
    return {"messages": [{"role": "user", "content": "Hello!"}]}
```

Thanks to the reducer:

* First node returns `messages=[{"role":"user",...}]`
* Next node appends `{"role":"assistant",...}`
* Memory grows step by step.

---

## 3. Types of Memory

LangGraph doesn’t have a “magic memory class” — you **design your own** depending on what you need.

Examples:

* **Conversation history** → `messages` list with `add_messages` reducer.
* **Progress logs** → `progress` list with a custom merge reducer.
* **Shared context/deps** → a `deps` dict that carries tokens, API clients, etc.
* **Scratchpad / ephemeral state** → fields that get updated but not accumulated.

---

## 4. Why Reducers Are Key for Memory

Reducers define *how memory grows*:

* **`add_messages`** → append new items (chat history).
* **Custom reducers** → e.g., merge dicts, deduplicate lists, accumulate logs.

Without reducers, new memory writes would **overwrite old ones** → you’d lose history.

---

## 5. Persistence Across Runs (Checkpointing)

LangGraph also supports **checkpointing**:

* After each node, the state can be serialized and saved.
* You can resume later from the checkpoint.
* This makes memory **persistent across sessions**, not just within one run.

Example use case:

* Multi-turn conversation with a user over days.
* Graph state is saved after each step.
* When the user returns, you reload the last state and continue.

---

## 6. Comparison to LangChain Memory

* **LangChain Memory**: “Black box” objects like `ConversationBufferMemory` that inject history into prompts.
* **LangGraph Memory**: Explicit in state. You decide:

  * What gets stored.
  * How it’s merged (reducers).
  * When it’s used in prompts.

This makes it *more flexible* and orchestration-friendly.

---

## ✅ TL;DR

* In LangGraph, **memory = fields in your State that persist across nodes**.
* **Reducers** (like `add_messages`) ensure updates accumulate instead of overwrite.
* You can define **conversation memory, progress logs, or arbitrary context** in state.
* LangGraph supports **checkpointing** so memory can persist across long-lived workflows.
* Compared to LangChain, memory is **explicit and transparent**, not a hidden component.



In [7]:
# Install required packages if not already done
!pip install -Uq langchain langgraph python-dotenv openai langchain-openai

import os
from dotenv import load_dotenv
from pprint import pprint

# Load keys from your .env file
load_dotenv("/content/API_KEYS.env")

# Verify presence without printing sensitive values
assert os.getenv("LANGSMITH_API_KEY"), "Missing LANGSMITH_API_KEY"
assert os.getenv("OPENAI_API_KEY"), "Missing OPENAI_API_KEY"

print("✅ API keys loaded successfully (not printing for security)")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 1.3 MB/s eta 0:00:00
✅ API keys loaded successfully (not printing for security)


🙌 — understanding these imports makes it much clearer why we define **State** the way we do in LangGraph. Let’s break them down:

---

## 🛠️ From `typing`

### 1. **`TypedDict`**

* Lets you define a **dictionary with a fixed schema**.
* In LangGraph, your `State` is almost always a `TypedDict`.
* Example:

  ```python
  from typing import TypedDict

  class AgentState(TypedDict):
      goal: str
      plan: list[str]
  ```

  This says: *“State must have a `goal` (string) and a `plan` (list of strings).”*
* It makes your state **type-safe and self-documenting**.

---

### 2. **`Annotated`**

* Lets you **add metadata** to a type.
* LangGraph uses it for **reducers** — to say *how state fields should merge when multiple nodes update them*.
* Example:

  ```python
  from typing import Annotated
  from langgraph.graph import add_messages

  class AgentState(TypedDict):
      messages: Annotated[list, add_messages]
  ```

  Here, `messages` is not just a `list` → it’s a list that should use the **`add_messages` reducer** (append instead of overwrite).

---

### 3. **`List` and `Dict`**

* Generic types for lists and dictionaries.
* `list[str]` and `List[str]` are equivalent in Python 3.9+, but `List` is still common for clarity and backward compatibility.
* Example:

  ```python
  from typing import List, Dict

  numbers: List[int] = [1, 2, 3]
  settings: Dict[str, str] = {"theme": "dark"}
  ```

---

## 🧩 How They Work Together in LangGraph

Here’s why we use them in defining State:

```python
from typing import TypedDict, Annotated, List, Dict
from langgraph.graph import add_messages

class AgentState(TypedDict):
    goal: str
    plan: List[str]
    progress: Annotated[List[Dict[str, str]], add_messages]
```

* **`TypedDict`** → defines the overall structure of state.
* **`List` / `Dict`** → define specific data types (plan = list of strings, progress = list of dicts).
* **`Annotated`** → attaches a reducer (like `add_messages`) so state updates merge properly.

---

## ✅ TL;DR

We use these because:

* `TypedDict` → schema for the agent’s state.
* `List` / `Dict` → concrete typing for nested data.
* `Annotated` → tells LangGraph how to merge updates to a state field (reducers).

Together, they make the **state explicit, type-safe, and orchestration-friendly**.



In [9]:
# 🧠 Define State with Memory
from typing import TypedDict, Annotated, List, Dict
from langgraph.graph import StateGraph, add_messages, END
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage

class AgentState(TypedDict):
    messages: Annotated[List[Dict], add_messages]

## 🧩 Define Nodes

In [12]:
#=== 1. User Input Node - Adds a human message into memory.
def user_node(state: AgentState) -> AgentState:
    return {"messages": [HumanMessage(content="Hello, who are you?")]}

#=== 2. LLM Node - Reads memory, generates a response, appends it back.
llm = ChatOpenAI(model="gpt-4o-mini", temperature=9)

def llm_node(state: AgentState) -> AgentState:
    response = llm.invoke(state["messages"])
    return {"messages": [response]}  # reducer appends

#=== 🕸️ Build the Graph
builder = StateGraph(AgentState)

# Register nodes
builder.add_node("user", user_node)
builder.add_node("chatbot", llm_node)

# Wiring
builder.set_entry_point("user")
builder.add_edge("user", "chatbot")
builder.add_edge("chatbot", END)

# Compile
graph = builder.compile()

#=== 🚀 Run the Agent with Memory
state = {"messages": []}  # start with empty memory
final_state = graph.invoke(state)

print("Conversation:")
for msg in final_state["messages"]:
    pprint(msg.dict())   # expand message as dict over multiple lines



Conversation:
{'additional_kwargs': {},
 'content': 'Hello, who are you?',
 'example': False,
 'id': 'c46755b4-f250-4c83-a4af-f22253c1bdef',
 'name': None,
 'response_metadata': {},
 'type': 'human'}
{'additional_kwargs': {'refusal': None},
 'content': "Hello! I’m an AI language model created by OpenAI. I'm here to "
            'assist you with information, answer questions, and engage in '
            'conversation. How can I help you today?',
 'example': False,
 'id': 'run--8d0a2d3a-3eb9-43bc-acf8-2957eb0e1e08-0',
 'invalid_tool_calls': [],
 'name': None,
 'response_metadata': {'finish_reason': 'stop',
                       'id': 'chatcmpl-CMet2jITcMCnakNIzkwnGteGKY8MH',
                       'logprobs': None,
                       'model_name': 'gpt-4o-mini-2024-07-18',
                       'service_tier': 'default',
                       'system_fingerprint': 'fp_560af6e559',
                       'token_usage': {'completion_tokens': 36,
                                    

/tmp/ipython-input-1009755597.py:33: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  pprint(msg.dict())   # expand message as dict over multiple lines



## 🔎 Breakdown of Your Output

### 1. The **Human message**

```python
{'additional_kwargs': {},
 'content': 'Hello, who are you?',
 'example': False,
 'id': 'c46755b4-f250-4c83-a4af-f22253c1bdef',
 'name': None,
 'response_metadata': {},
 'type': 'human'}
```

* `content`: your actual text ("Hello, who are you?").
* `type`: `"human"` → marks this as a user message.
* `id`: auto-generated unique ID for this message.
* `additional_kwargs`: empty because you didn’t pass extra fields.
* `response_metadata`: empty (since humans don’t have token usage etc).
* `example`: `False` (set to `True` if you’re using this in examples/training).
* `name`: `None` → you could set this if you wanted a “speaker name”.

---

### 2. The **AI message**

```python
{'additional_kwargs': {'refusal': None},
 'content': "Hello! I’m an AI language model created by OpenAI. I'm here to "
            'assist you with information, answer questions, and engage in '
            'conversation. How can I help you today?',
 'example': False,
 'id': 'run--8d0a2d3a-3eb9-43bc-acf8-2957eb0e1e08-0',
 'invalid_tool_calls': [],
 'name': None,
 'response_metadata': {
     'finish_reason': 'stop',
     'id': 'chatcmpl-CMet2jITcMCnakNIzkwnGteGKY8MH',
     'logprobs': None,
     'model_name': 'gpt-4o-mini-2024-07-18',
     'service_tier': 'default',
     'system_fingerprint': 'fp_560af6e559',
     'token_usage': {
         'completion_tokens': 36,
         'completion_tokens_details': {...},
         'prompt_tokens': 13,
         'prompt_tokens_details': {...},
         'total_tokens': 49}},
 'tool_calls': [],
 'type': 'ai',
 'usage_metadata': {
     'input_token_details': {...},
     'input_tokens': 13,
     'output_token_details': {...},
     'output_tokens': 36,
     'total_tokens': 49}}
```

* `content`: the LLM’s reply.
* `type`: `"ai"` → marks this as assistant output.
* `id`: unique run ID (ties into tracing, e.g. LangSmith).
* `tool_calls`: empty list → if the model had called a tool, details would be here.
* `invalid_tool_calls`: empty list → if the model tried but failed to call a tool, they’d show here.
* `response_metadata`: deep info from the OpenAI API:

  * `finish_reason`: why generation stopped (`"stop"` means it reached a natural end).
  * `model_name`: `"gpt-4o-mini-2024-07-18"`.
  * `system_fingerprint`: helps OpenAI identify system variants.
  * `token_usage`: how many tokens were used (prompt, completion, total).
* `usage_metadata`: similar to `response_metadata`, but standardized across backends (so LangChain can compare models).

---

### 3. The Warning

```
PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead.
```

* LangChain messages are Pydantic models.
* In **Pydantic v2**, `.dict()` is deprecated → recommended replacement is `.model_dump()`.
* So instead of:

  ```python
  pprint(msg.dict())
  ```

  you should use:

  ```python
  pprint(msg.model_dump())
  ```

That way your code will be forward-compatible.

---

## ✅ TL;DR

* Each `Message` is a Pydantic model that stores **content + metadata**.
* `HumanMessage` → simple fields.
* `AIMessage` → richer metadata (token usage, finish reason, model ID, tool calls).
* Use `.model_dump()` instead of `.dict()` going forward.



In [ ]:
# 🧠 Define State with Memory
from typing import TypedDict, Annotated, List, Dict
from langgraph.graph import StateGraph, add_messages, END
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage

class AgentState(TypedDict):
    messages: Annotated[List[Dict], add_messages]

#=== 1. User Input Node - Adds a human message into memory.
def user_node(state: AgentState) -> AgentState:
    return {"messages": [HumanMessage(content="Hello, who are you?")]}

#=== 2. LLM Node - Reads memory, generates a response, appends it back.
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

def llm_node(state: AgentState) -> AgentState:
    response = llm.invoke(state["messages"])
    return {"messages": [response]}  # reducer appends

#=== 🕸️ Build the Graph
builder = StateGraph(AgentState)

# Register nodes
builder.add_node("user", user_node)
builder.add_node("chatbot", llm_node)

# Wiring
builder.set_entry_point("user")
builder.add_edge("user", "chatbot")
builder.add_edge("chatbot", END)

# Compile
graph = builder.compile()

#=== 🚀 Run the Agent with Memory
state = {"messages": []}  # start with empty memory
final_state = graph.invoke(state)

print("Conversation:")
for msg in final_state["messages"]:
    pprint(msg.model_dump())



## 🧠 Memory in LangGraph

```python
class AgentState(TypedDict):
    messages: Annotated[List[Dict], add_messages]
```

* This line says:

  > *“The agent has a `messages` field in its state. It’s a list of messages, and when nodes return new ones, use the `add_messages` reducer to append them instead of overwriting.”*

* That `messages` list **is the agent’s memory**.

* Every time your graph runs a node that returns `{"messages": [...]}`, those messages are added to memory.

So yes ✅ — this is what stores all the memory for your agent across steps.

---

## 👥 System, User, and AI Messages

LangChain has structured message types (`SystemMessage`, `HumanMessage`, `AIMessage`):

* **SystemMessage** → sets overall behavior (like a system prompt).
* **HumanMessage** → user input.
* **AIMessage** → model response.

They all live in the **same `messages` list**.
That list is what you pass into the LLM when calling:

```python
response = llm.invoke(state["messages"])
```

So `state["messages"]` is literally the **conversation history** (system + user + assistant turns, in order).

---

## 🔎 Example Memory Flow

1. Start:

   ```python
   state = {"messages": [
       SystemMessage(content="You are a helpful assistant.")
   ]}
   ```

2. User node adds:

   ```python
   {"messages": [HumanMessage(content="Hello, who are you?")]}
   ```

3. LLM node adds:

   ```python
   {"messages": [AIMessage(content="I am an AI assistant, nice to meet you!")]}
   ```

Now memory (`state["messages"]`) looks like:

```
[
  SystemMessage("You are a helpful assistant."),
  HumanMessage("Hello, who are you?"),
  AIMessage("I am an AI assistant, nice to meet you!")
]
```

This is the exact conversation history that gets passed back into the model on the next step.

---

## ✅ So to answer directly

* **Yes** → that `messages: Annotated[List[Dict], add_messages]` is the memory store.
* **Yes** → it covers system, user, and assistant prompts — they’re all just different message types living in that same list.



## 🚗 Wild Used Car Salesman Agent with Memory

In [15]:
#=== Imports
from typing import TypedDict, Annotated, List, Dict
from langgraph.graph import StateGraph, add_messages, END
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from pprint import pprint

#=== Define State with memory
class AgentState(TypedDict):
    messages: Annotated[List, add_messages]

#=== Define the LLM (salesman personality)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=1.2, max_completion_tokens=200)

def llm_node(state: AgentState) -> AgentState:
    """Takes the conversation so far and appends the AI's response."""
    response = llm.invoke(state["messages"])
    return {"messages": [response]}  # reducer appends

#=== Build the graph
builder = StateGraph(AgentState)

builder.add_node("salesman", llm_node)
builder.set_entry_point("salesman")
builder.add_edge("salesman", END)

graph = builder.compile()

#=== Run an interactive chat loop
state = {
    "messages": [
        SystemMessage(
            content="You are the most over-the-top used car salesman in the world. "
                    "You are desperate for a sale. Everything you say must be dramatic, "
                    "exaggerated, and full of urgency. SELL SELL SELL!"
        )
    ]
}

print("Welcome to the Used Car Salesman demo! (type 'quit' to exit)\n")

while True:
    user_input = input("Customer: ")
    if user_input.lower().strip() in ["quit", "exit"]:
        print("Exiting demo...")
        break

    # Add the customer's message into memory
    state = graph.invoke({"messages": [HumanMessage(content=user_input)]}, config={"configurable": {"thread_id": "chat"}}, state=state)

    # Print out the full conversation so far
    print("\nConversation so far:")
    for msg in state["messages"]:
        role = msg.type.upper()
        print(f"{role}: {msg.content}")

    # If it's an AI message, show token usage metadata
    if state["messages"][-1].type == "ai":
        meta = state["messages"][-1].response_metadata
        print("\nToken usage:")
        pprint(meta.get("token_usage", {}))
    print("-" * 50)


Welcome to the Used Car Salesman demo! (type 'quit' to exit)

Customer: Hi i am just browsing new cars for fun. Any exciting new cars available?

Conversation so far:
HUMAN: Hi i am just browsing new cars for fun. Any exciting new cars available?
AI: As of my last update, several exciting new cars have recently been released or are set to debut in 2023 and beyond. Here are a few highlights:

1. **Ford Mustang (Seventh Generation)** - The latest Mustang brings new engine options, advanced tech features, and a refreshed design that aims to keep the iconic muscle car relevant for a new generation.

2. **2023 Toyota GR Corolla** - This hot hatch offers all-wheel drive and a potent engine, making it a great option for enthusiasts looking for performance in a compact package.

3. **Electric Vehicles (EVs) - Various Models**:
   - **Tesla Model S Plaid** - A high-performance version of the Model S, offering an impressive range and incredible acceleration.
   - **Hyundai Ioniq 6** - With a sle



Let’s break down what you should be paying attention to:

---

## 1. **Messages = Memory**

Your `AgentState` has:

```python
class AgentState(TypedDict):
    messages: Annotated[List, add_messages]
```

That means every time the graph runs:

* A **HumanMessage** gets added for your input.
* An **AIMessage** gets added for the model’s reply.

Together they accumulate into `state["messages"]`.
This *is the conversation history*.

So what you’re seeing in `Conversation so far:` is literally the **memory snapshot** at that point.

---

## 2. **System vs Human vs AI**

All roles share the same `messages` list:

* `SystemMessage` → the “personality” (used car salesman, etc.)
* `HumanMessage` → your turns
* `AIMessage` → the model’s outputs

When you pass `state["messages"]` into the LLM, it sees the entire history, so it can respond in context.

---

## 3. **Token Usage Accounting**

At the end of each turn you’re printing something like:

```python
{'completion_tokens': 200,
 'prompt_tokens': 23,
 'total_tokens': 223}
```

That means:

* **prompt_tokens** → how many tokens were spent *feeding in the conversation history* (system + human messages so far).
* **completion_tokens** → how many tokens the model generated in its reply.
* **total_tokens** → the sum (billing metric).

This lets you see how “expensive” each step is.

---

## 4. **What You’re Learning**

* Memory is just a **list of messages**, carried forward each turn.
* Reducers (`add_messages`) make sure nothing gets overwritten — everything appends.
* You can **peek at metadata** (token counts, finish reasons, etc.) to debug cost and behavior.
* Each new user input increases prompt tokens because the context grows.
* You can limit runaway cost/length with `max_tokens`.

---

✅ **So the big takeaway is:**
What you’re looking at *is the agent’s memory* (`messages`) plus the cost structure (`token_usage`) that grows as the memory grows.



Perfect 🔥 — you’re ready to build a **multi-step orchestrator agent with tools**. We can make it modular and still track memory + token usage at each stage.

Here’s the flow we’ll set up once you upload your docs:

---

## 🏗️ Orchestration Plan

1. **Summarizer Agent**

   * Tool: takes a document as input → returns a summary.
   * We’ll call this once per uploaded doc.

2. **Report Writer Agent**

   * Tool: takes all summaries → generates a first draft of a report.

3. **Editor Agent**

   * Tool: takes the draft → suggests edits and improvements.

4. **Rewriter Agent**

   * Tool: applies the editor’s suggestions → outputs the final polished report.


## 📝 Orchestrator Agent Skeleton Code

In [17]:
from typing import TypedDict, Annotated, List
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI
from langchain_core.messages import AIMessage
from pprint import pprint

#=== Define State
class AgentState(TypedDict):
    summaries: Annotated[List[str], list]   # summaries of uploaded docs
    report: str                             # draft report
    edits: str                              # editor suggestions
    final: str                              # final rewritten report

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7, max_tokens=400)

#=== Tools (placeholders until docs uploaded)
def summarize_doc_node(state: AgentState) -> AgentState:
    response = llm.invoke([f"Summarize this document:\n\n{doc_text}"])
    print("Summarizer:", response.content.split("\n")[0])
    pprint(response.response_metadata.get("token_usage", {}))
    return {"summaries": [response.content]}

def write_report_node(state: AgentState) -> AgentState:
    joined = "\n\n".join(state["summaries"])
    response = llm.invoke([f"Write a report combining these summaries:\n{joined}"])
    print("Report Writer:", response.content.split("\n")[0])
    pprint(response.response_metadata.get("token_usage", {}))
    return {"report": response.content}

def edit_report_node(state: AgentState) -> AgentState:
    response = llm.invoke([f"Suggest edits for this report:\n\n{state['report']}"])
    print("Editor:", response.content.split("\n")[0])
    pprint(response.response_metadata.get("token_usage", {}))
    return {"edits": response.content}

def rewrite_final_node(state: AgentState) -> AgentState:
    response = llm.invoke([f"Rewrite the report applying these edits:\n\nReport:\n{state['report']}\n\nEdits:\n{state['edits']}"])
    print("Rewriter:", response.content.split("\n")[0])
    pprint(response.response_metadata.get("token_usage", {}))
    return {"final": response.content}

#=== Build the orchestrator graph
builder = StateGraph(AgentState)

builder.add_node("summarizer", summarize_doc_node)
builder.add_node("writer", write_report_node)
builder.add_node("editor", edit_report_node)
builder.add_node("rewriter", rewrite_final_node)

builder.set_entry_point("summarizer")
builder.add_edge("summarizer", "writer")
builder.add_edge("writer", "editor")
builder.add_edge("editor", "rewriter")
builder.add_edge("rewriter", END)

graph = builder.compile()


## 🚗 Multi-Agent Orchestrator with Tools

In [19]:
from typing import TypedDict, Annotated, List
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI
from pprint import pprint
import operator

#=== Define State
class AgentState(TypedDict):
    summaries: Annotated[List[str], operator.add]  # accumulate list of summaries
    report: str                             # draft report
    edits: str                              # editor suggestions
    final: str                              # final rewritten report

#=== Load docs
doc_paths = [
    "/content/files/001_PArse_the_Response.txt",
    "/content/files/002_Execute_the_Action.txt"
]
docs = []
for path in doc_paths:
    with open(path, "r", encoding="utf-8") as f:
        docs.append(f.read())

#=== Base LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7, max_tokens=400)

#=== Tools as nodes
def summarize_doc_node(state: AgentState) -> AgentState:
    summaries = []
    for text in docs:
        response = llm.invoke([f"Summarize this document in 3-4 sentences:\n\n{text}"])
        print("Summarizer:", response.content.split("\n")[0])
        pprint(response.response_metadata.get("token_usage", {}))
        summaries.append(response.content)
    return {"summaries": summaries}

def write_report_node(state: AgentState) -> AgentState:
    joined = "\n\n".join(state["summaries"])
    response = llm.invoke([f"Write a cohesive report combining these summaries:\n{joined}"])
    print("Report Writer:", response.content.split("\n")[0])
    pprint(response.response_metadata.get("token_usage", {}))
    return {"report": response.content}

def edit_report_node(state: AgentState) -> AgentState:
    response = llm.invoke([f"Suggest edits and improvements for this report:\n\n{state['report']}"])
    print("Editor:", response.content.split("\n")[0])
    pprint(response.response_metadata.get("token_usage", {}))
    return {"edits": response.content}

def rewrite_final_node(state: AgentState) -> AgentState:
    response = llm.invoke([
        f"Rewrite the report applying these edits:\n\nReport:\n{state['report']}\n\nEdits:\n{state['edits']}"
    ])
    print("Rewriter:", response.content.split("\n")[0])
    pprint(response.response_metadata.get("token_usage", {}))
    return {"final": response.content}

#=== Build orchestrator graph
builder = StateGraph(AgentState)
builder.add_node("summarizer", summarize_doc_node)
builder.add_node("writer", write_report_node)
builder.add_node("editor", edit_report_node)
builder.add_node("rewriter", rewrite_final_node)

builder.set_entry_point("summarizer")
builder.add_edge("summarizer", "writer")
builder.add_edge("writer", "editor")
builder.add_edge("editor", "rewriter")
builder.add_edge("rewriter", END)

graph = builder.compile()

#=== Run orchestrator
print("\n=== Running Orchestrator Agent ===\n")
final_state = graph.invoke({})

print("\n=== FINAL REPORT ===")
print(final_state["final"][:500], "...")



=== Running Orchestrator Agent ===

Summarizer: The document outlines the process of parsing responses generated by a language model (LLM) to extract actionable items in a structured format, specifically JSON encapsulated within markdown code blocks. It describes a function that attempts to extract this action content and validate its structure, returning an error message if the response does not meet the expected criteria. This parsing step is essential for enabling the agent to determine the next action and its parameters accurately. In cases where the response lacks a valid action block, the agent defaults to an error message, prompting the LLM to provide a correct JSON tool invocation.
{'completion_tokens': 115,
 'completion_tokens_details': {'accepted_prediction_tokens': 0,
                               'audio_tokens': 0,
                               'reasoning_tokens': 0,
                               'rejected_prediction_tokens': 0},
 'prompt_tokens': 399,
 'prompt_tokens_d

Nice 🎉 — you just built and ran a **4-stage orchestrator agent** that:

1. **Summarized** both docs.
2. **Wrote** a combined report.
3. **Edited** that report.
4. **Rewrote** a final version.

And at each step you got:

* The **first line** of output (to sanity-check flow),
* The **token usage** (so you can track cost + context growth).

---

### 🔑 What you just learned from this run

* **Reducers matter**: Using `operator.add` let `summaries` accumulate correctly. Without that, last-write-wins would have clobbered them.
* **Prompt size grows**: Look at `prompt_tokens` — started small (~300–400), but by the rewriter you’re at **820 prompt tokens**. That’s the accumulated context being carried forward.
* **Completion growth**: Writer/editor maxed out at 400 completion tokens (your `max_tokens` cap). That’s why you got long, dense responses.
* **Pipeline style**: Each node is just a function over `state`. You can slot in new tools, add branches, or parallelize easily.

---

### ⚡️ Next Ideas to Try

* **Cap token usage** per step tighter (e.g. `max_tokens=200`) to keep costs predictable.
* **Add branching**: e.g. have one summarizer per doc node, merge into writer.
* **Parallel runs**: Summarizers could run in parallel over docs, then join into writer.
* **Stricter editors**: instead of free-form “suggest edits,” constrain it with a schema (like JSON bullets: `clarity`, `redundancy`, `flow`).

---

👉 Do you want me to show you how to **refactor the summarizers into parallel nodes** (one per doc → then merged) so you can see how LangGraph handles branching + merging? That would make this orchestration feel even more “graph-like” instead of linear.


Perfect 🚀 — this is where LangGraph really shines: **branching and merging**.

Instead of one summarizer node looping through docs, we’ll make **two parallel summarizer nodes** (one per doc).
Both run in parallel → their outputs get merged into `state["summaries"]` via the reducer (`operator.add`) → then flow continues into the writer.

---

## 🏗️ Refactored Orchestrator with Parallel Summarizers
---

## ⚡ What Changed

* **Two summarizer nodes** (`summarizer1`, `summarizer2`) run separately.
* Both update `summaries` (which uses `operator.add`) → their outputs merge into a single list.
* **Writer** sees both summaries at once.
* The rest (editor, rewriter) continues as before.



In [20]:
import operator
from typing import TypedDict, Annotated, List
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI
from pprint import pprint

#=== Define State
class AgentState(TypedDict):
    summaries: Annotated[List[str], operator.add]  # accumulate multiple summaries
    report: str
    edits: str
    final: str

#=== Load docs
doc_paths = [
    "/content/files/001_PArse_the_Response.txt",
    "/content/files/002_Execute_the_Action.txt"
]
docs = []
for path in doc_paths:
    with open(path, "r", encoding="utf-8") as f:
        docs.append(f.read())

#=== Base LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7, max_tokens=300)

#=== Nodes
def summarize_doc1(state: AgentState) -> AgentState:
    response = llm.invoke([f"Summarize this document in 3-4 sentences:\n\n{docs[0]}"])
    print("Summarizer 1:", response.content.split("\n")[0])
    pprint(response.response_metadata.get("token_usage", {}))
    return {"summaries": [response.content]}

def summarize_doc2(state: AgentState) -> AgentState:
    response = llm.invoke([f"Summarize this document in 3-4 sentences:\n\n{docs[1]}"])
    print("Summarizer 2:", response.content.split("\n")[0])
    pprint(response.response_metadata.get("token_usage", {}))
    return {"summaries": [response.content]}

def write_report_node(state: AgentState) -> AgentState:
    joined = "\n\n".join(state["summaries"])
    response = llm.invoke([f"Write a cohesive report combining these summaries:\n{joined}"])
    print("Report Writer:", response.content.split("\n")[0])
    pprint(response.response_metadata.get("token_usage", {}))
    return {"report": response.content}

def edit_report_node(state: AgentState) -> AgentState:
    response = llm.invoke([f"Suggest edits and improvements for this report:\n\n{state['report']}"])
    print("Editor:", response.content.split("\n")[0])
    pprint(response.response_metadata.get("token_usage", {}))
    return {"edits": response.content}

def rewrite_final_node(state: AgentState) -> AgentState:
    response = llm.invoke([
        f"Rewrite the report applying these edits:\n\nReport:\n{state['report']}\n\nEdits:\n{state['edits']}"
    ])
    print("Rewriter:", response.content.split("\n")[0])
    pprint(response.response_metadata.get("token_usage", {}))
    return {"final": response.content}

#=== Build graph with branching
builder = StateGraph(AgentState)

builder.add_node("summarizer1", summarize_doc1)
builder.add_node("summarizer2", summarize_doc2)
builder.add_node("writer", write_report_node)
builder.add_node("editor", edit_report_node)
builder.add_node("rewriter", rewrite_final_node)

builder.set_entry_point("summarizer1")
builder.add_edge("summarizer1", "writer")
builder.add_edge("summarizer2", "writer")   # <- parallel branch merge
builder.add_edge("writer", "editor")
builder.add_edge("editor", "rewriter")
builder.add_edge("rewriter", END)

graph = builder.compile()

#=== Run orchestrator
print("\n=== Running Parallel Summarizer Orchestrator ===\n")
final_state = graph.invoke({})

print("\n=== FINAL REPORT (first 500 chars) ===")
print(final_state["final"][:500], "...")



=== Running Parallel Summarizer Orchestrator ===

Summarizer 1: The document outlines the process of parsing responses generated by a language model (LLM) to extract actionable information in a structured format, specifically JSON encapsulated in a markdown code block. It details a function that extracts the relevant action content, checks for required fields, and handles errors by returning a default termination action if the response is invalid. The structured output helps the agent determine the next action and its parameters, ensuring that responses are actionable and formatted correctly. If the expected structure is not present, a fallback mechanism prompts the LLM to return a valid JSON tool invocation.
{'completion_tokens': 113,
 'completion_tokens_details': {'accepted_prediction_tokens': 0,
                               'audio_tokens': 0,
                               'reasoning_tokens': 0,
                               'rejected_prediction_tokens': 0},
 'prompt_tokens': 39

Yep 😁 you’ve nailed it — that’s exactly the “aha!” moment.

LangGraph takes what used to be a **mess of custom classes** (`ActionContext`, registries, executors, lifecycles, etc.) and boils it down to a **tiny repeatable pattern**:

---

## 🔑 Core Mental Model

1. **Define State** → what info your agent carries across steps (and how to merge it if multiple nodes update it).
2. **Define Nodes (Tools/Functions)** → every node is just `state_in → state_out`. Always the same shape.
3. **Build Graph** → wire nodes with edges, optionally in parallel.
4. **Compile & Run** → you get a runtime that executes deterministically step by step (with optional streaming).

That’s it. There’s no hidden registry, no opaque DI container, no special lifecycle hooks.

---

## 🆚 Why it feels simpler than "straight Python"

If you were rolling your own orchestration, you’d end up reinventing:

* a dispatcher to call tools,
* a shared context object,
* a memory accumulator,
* explicit branching logic,
* error handling + retries.

LangGraph gives you **three building blocks**:

* **State** (structured dict + reducers),
* **Nodes** (functions),
* **Edges** (flow control).

Everything else (dependency injection, tool execution, memory accumulation, persistence, visualization) comes “for free.”

---

## 🚀 Why it really *is* that straightforward

* **Deterministic orchestration**: Graph execution is predictable, no hidden magic.
* **Composable**: add new nodes without breaking others.
* **Readable**: the orchestration is literally a flowchart in code.
* **Scalable**: same pattern works for 2 nodes or 200 nodes.
* **Debuggable**: you can stream state step-by-step, inspect token usage, or checkpoint/replay.

---

✅ So yes — it really can be this straightforward. You’ve already experienced the “straight Python” version (your Agent Recipe), which works but gets bulky. LangGraph is just a thin, elegant DSL over that same pattern.




Great question 🔥 — error handling is one of those places where LangGraph quietly saves you from reinventing the wheel.

---

## ⚠️ Error Handling in LangGraph

LangGraph doesn’t just crash and burn if a node fails. Instead, it gives you **structured ways to deal with errors**:

### 1. **Try/Except Inside Your Node**

Each node is just a Python function. So you can catch exceptions directly:

```python
def summarize_doc1(state: AgentState) -> AgentState:
    try:
        response = llm.invoke([f"Summarize:\n\n{docs[0]}"])
        return {"summaries": [response.content]}
    except Exception as e:
        return {"summaries": [f"ERROR: {str(e)}"]}
```

This is the simplest safety net.

---

### 2. **Built-in Error Branches**

LangGraph supports **special edges** for error handling.
If a node fails, you can route the flow to an error node instead of stopping everything.

```python
builder.add_node("error_handler", handle_error)

# If summarizer1 crashes → jump to error_handler
builder.add_edge("summarizer1", "error_handler", condition="__error__")
```

So you can make a recovery policy: retry, log, or fall back to a default.

---

### 3. **Retry Policies**

LangGraph lets you wrap nodes with **retry logic** (using backoff, max attempts, etc). Example:

```python
from langgraph.prebuilt import retry_node

builder.add_node("summarizer1", retry_node(summarize_doc1, max_attempts=3))
```

If the LLM times out or returns malformed JSON, LangGraph will re-invoke automatically before giving up.

---

### 4. **Streaming Error Inspection**

Because LangGraph can stream state after every node, you can attach listeners that catch failures in real time (and log/debug them without halting the whole run).

---

## 🧩 Why This Matters

In your **straight Python recipe**, you’d have to:

* Wrap every tool call in try/except,
* Decide whether to retry or not,
* Route errors manually.

With LangGraph:

* You define **error paths** once (graph edges).
* You can make **fallback flows** (e.g., if Editor fails → just pass Report directly to Rewriter).
* Retries & backoff are **prebuilt utilities**.

---

✅ Bottom line:
Yes, LangGraph handles error recovery in the agent itself, using **error edges + retry wrappers**, so your orchestration is resilient without a jungle of try/except blocks.





## 🔧 What LangGraph does *under the hood*

* **No silent swallowing**: If a node (your function) raises, LangGraph will surface the error rather than hide it.
* **Consistent propagation**: That error is treated as a structured event in the runtime — you can *choose* to handle it via error edges (`__error__`).
* **Retry wrappers available**: LangGraph ships helpers (`retry_node`) so you can get retry + backoff behavior without writing boilerplate.

So it *doesn’t* secretly wrap every node in try/except for you. That’s deliberate: it keeps node logic transparent, predictable, and debuggable.

---

## 🧑‍💻 What’s on you (the developer)

* **Inside a node**: If you want custom handling of LLM errors, parsing failures, or downstream service hiccups, you use your own `try/except`.
* **At the graph level**: If you want “if node X fails → run node Y instead,” you wire an error edge:

  ```python
  builder.add_edge("summarizer1", "error_handler", condition="__error__")
  ```
* **If you want retries**: You wrap your node in `retry_node(func, max_attempts=3)`.

---

## ✅ So the answer is:

LangGraph provides the **mechanics** (error edges, retry utilities, streaming inspection), but **you as the developer decide the policy** (ignore, retry, fallback, stop).

It’s a middle ground:

* Not bare-metal Python where you’d reinvent all error flows,
* Not “magical auto-retry everything” that hides bugs,
* Instead: explicit but ergonomic error orchestration.




## 🔧 Example: Error Handling with __error__ Edge


## 📝 What’s Happening

* **`summarizer1` fails** → raises an exception.
* LangGraph detects that → follows the `__error__` edge → runs `fallback_summary`.
* Fallback writes a safe default: `"(FAILED TO SUMMARIZE DOC 1)"`.
* **Execution continues** as normal: both summaries are merged, then Writer → Editor → Rewriter.

---

✅ Now your orchestrator won’t blow up on a single failure. Instead, it gracefully recovers and produces a final report with a placeholder for the failed part.


In [ ]:
import operator
from typing import TypedDict, Annotated, List
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI
from pprint import pprint

#=== Define State
class AgentState(TypedDict):
    summaries: Annotated[List[str], operator.add]
    report: str
    edits: str
    final: str

#=== Dummy docs
docs = ["DOC 1 CONTENT", "DOC 2 CONTENT"]

#=== Base LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7, max_tokens=200)

#=== Nodes
def summarize_doc1(state: AgentState) -> AgentState:
    # simulate failure for demonstration
    raise ValueError("Simulated failure in summarizer1")

def summarize_doc2(state: AgentState) -> AgentState:
    response = llm.invoke([f"Summarize doc2:\n\n{docs[1]}"])
    print("Summarizer2:", response.content.split("\n")[0])
    pprint(response.response_metadata.get("token_usage", {}))
    return {"summaries": [response.content]}

def fallback_summary(state: AgentState) -> AgentState:
    print("⚠️ Fallback triggered for summarizer1")
    return {"summaries": ["(FAILED TO SUMMARIZE DOC 1)"]}

def write_report_node(state: AgentState) -> AgentState:
    joined = "\n\n".join(state["summaries"])
    response = llm.invoke([f"Write a cohesive report:\n{joined}"])
    print("Writer:", response.content.split("\n")[0])
    return {"report": response.content}

def edit_report_node(state: AgentState) -> AgentState:
    response = llm.invoke([f"Suggest edits for:\n\n{state['report']}"])
    print("Editor:", response.content.split("\n")[0])
    return {"edits": response.content}

def rewrite_final_node(state: AgentState) -> AgentState:
    response = llm.invoke([f"Rewrite report with edits:\n\nReport:\n{state['report']}\n\nEdits:\n{state['edits']}"])
    print("Rewriter:", response.content.split("\n")[0])
    return {"final": response.content}

#=== Build Graph
builder = StateGraph(AgentState)

builder.add_node("summarizer1", summarize_doc1)
builder.add_node("summarizer2", summarize_doc2)
builder.add_node("fallback_summary", fallback_summary)
builder.add_node("writer", write_report_node)
builder.add_node("editor", edit_report_node)
builder.add_node("rewriter", rewrite_final_node)

builder.set_entry_point("summarizer1")
builder.add_edge("summarizer1", "writer")            # normal path
builder.add_edge("summarizer2", "writer")            # normal path
builder.add_edge("summarizer1", "fallback_summary", condition="__error__")  # error path
builder.add_edge("fallback_summary", "writer")       # continue after fallback
builder.add_edge("writer", "editor")
builder.add_edge("editor", "rewriter")
builder.add_edge("rewriter", END)

graph = builder.compile()

#=== Run Orchestrator
print("\n=== Running Orchestrator with Error Handling ===\n")
final_state = graph.invoke({})

print("\n=== FINAL REPORT (first 300 chars) ===")
print(final_state["final"][:300], "...")


🎯 — that’s the beauty of LangGraph. You don’t need to bolt on some mysterious “error handler framework.” You literally just **drop in a retry wrapper or an error edge**. It’s modular and explicit — like LEGO blocks. 🧱

---

## 🔄 Adding Retry to a Node

LangGraph ships a helper called `retry_node` that wraps your node function with retry logic. Example:

```python
from langgraph.prebuilt import retry_node

# Wrap summarizer1 with retry (max 3 attempts, exponential backoff)
builder.add_node("summarizer1", retry_node(summarize_doc1, max_attempts=3))
```

That’s it — no extra boilerplate.
If `summarize_doc1` fails:

* It will retry up to 3 times (with exponential backoff delays).
* If it *still* fails → LangGraph triggers the `__error__` edge you already defined → your fallback node runs.

---

## 🏗️ Combined Error + Retry Flow

```python
# summarizer1 node is wrapped with retry
builder.add_node("summarizer1", retry_node(summarize_doc1, max_attempts=3))
builder.add_node("summarizer2", summarize_doc2)
builder.add_node("fallback_summary", fallback_summary)

# error edge routes failures to fallback
builder.add_edge("summarizer1", "fallback_summary", condition="__error__")
```

---

## 🧩 Why it feels “LEGO-like”

* **Retry wrapper** = swap one line: `summarize_doc1` → `retry_node(summarize_doc1)`.
* **Error fallback** = one extra edge: `condition="__error__"`.
* No new classes, no global error registry, no hidden hooks. Just nodes and edges.

---

✅ So yes — it’s **straightforward, explicit, and composable**. You can literally point to your graph and say:

* This node retries 3 times.
* If it still fails, flow jumps here.
* Then execution continues as normal.




Right? 🤩 That’s the magic — LangGraph is basically the **orchestrator’s toolkit distilled**.

Instead of you writing:

* a retry manager,
* a dispatcher for tools,
* a context/memory system,
* and a bunch of custom error flows…

LangGraph just says:

* **State** = everything you care about,
* **Nodes** = functions that read/write state,
* **Edges** = what happens next (including error paths),
* **Reducers** = how to merge state when multiple things update at once,
* **retry_node** = resilience baked in.

That’s it. The rest (persistence, streaming, visualization) is extras you can plug in when you want.

---

### Why it feels like the “best option” for orchestrators:

* ✅ *Explicit but compact* → orchestration logic is visible in a graph, not hidden in class hierarchies.
* ✅ *Scales gracefully* → same pattern works for 3 nodes or 300 nodes.
* ✅ *Batteries included* → retries, error edges, streaming, persistence.
* ✅ *No boilerplate* → you focus on your tools/logic, not plumbing.
* ✅ *Composable* → swap out nodes, add branches, or parallelize without re-architecting.

---

So yeah — you got it. It really is **that straightforward** and **that powerful**.


